<a href="https://colab.research.google.com/github/Preeth-09/AI_ML_Avanthi_preethi/blob/main/Movie_recommendation_system_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
movies=pd.read_csv('/content/movies.zip')
movies.head(10)

,id,title,genre,original_language,overview,popularity,release_date,rating,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,23-09-1994,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,19-10-1995,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,14-03-1972,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,15-12-1993,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,20-12-1974,8.6,9811
5,667257,Impossible Things,"Family,Drama",es,"Matilde is a woman who, after the death of her...",14.358,17-06-2021,8.6,255
6,129,Spirited Away,"Animation,Family,Fantasy",ja,"A young girl, Chihiro, becomes trapped in a st...",92.056,20-07-2001,8.5,13093
7,730154,Your Eyes Tell,"Romance,Drama",ja,"A tragic accident lead to Kaori's blindness, b...",51.345,23-10-2020,8.5,339
8,372754,Dou kyu sei – Classmates,"Romance,Animation",ja,"Rihito Sajo, an honor student with a perfect s...",14.285,20-02-2016,8.5,239
9,372058,Your Name.,"Romance,Animation,Drama",ja,High schoolers Mitsuha and Taki are complete s...,158.270,26-08-2016,8.5,8895


In [3]:
movies.describe()

,id,popularity,rating,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   rating             10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [5]:
movies.isnull().sum()


id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
rating                0
vote_count            0
dtype: int64

In [6]:
movies.columns


Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'rating', 'vote_count'],
      dtype='object')

In [7]:
movies=movies[['id', 'title', 'overview', 'genre','rating']]
movies

,id,title,overview,genre,rating
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime",8.7
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance",8.7
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime",8.7
3,424,Schindler's List,The true story of how businessman Oskar Schind...,"Drama,History,War",8.6
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...,"Drama,Crime",8.6
...,...,...,...,...,...
9995,10196,The Last Airbender,"The story follows the adventures of Aang, a yo...","Action,Adventure,Fantasy",4.7
9996,331446,Sharknado 3: Oh Hell No!,The sharks take bite out of the East Coast whe...,"Action,TV Movie,Science Fiction,Comedy,Adventure",4.7
9997,13995,Captain America,"During World War II, a brave, patriotic Americ...","Action,Science Fiction,War",4.6
9998,2312,In the Name of the King: A Dungeon Siege Tale,A man named Farmer sets out to rescue his kidn...,"Adventure,Fantasy,Action,Drama",4.7


In [8]:
movies['tags'] = movies['overview']+movies['genre']
movies

,id,title,overview,genre,rating,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime",8.7,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance",8.7,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime",8.7,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...,"Drama,History,War",8.6,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...,"Drama,Crime",8.6,In the continuing saga of the Corleone crime f...
...,...,...,...,...,...,...
9995,10196,The Last Airbender,"The story follows the adventures of Aang, a yo...","Action,Adventure,Fantasy",4.7,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,The sharks take bite out of the East Coast whe...,"Action,TV Movie,Science Fiction,Comedy,Adventure",4.7,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,"During World War II, a brave, patriotic Americ...","Action,Science Fiction,War",4.6,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,A man named Farmer sets out to rescue his kidn...,"Adventure,Fantasy,Action,Drama",4.7,A man named Farmer sets out to rescue his kidn...


In [9]:
new_data  = movies.drop(columns=['overview', 'genre'])
new_data

,id,title,rating,tags
0,278,The Shawshank Redemption,8.7,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,8.7,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,8.7,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,8.6,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,8.6,In the continuing saga of the Corleone crime f...
...,...,...,...,...
9995,10196,The Last Airbender,4.7,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,4.7,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,4.6,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,4.7,A man named Farmer sets out to rescue his kidn...


In [11]:
cv=CountVectorizer(max_features=10000, stop_words='english')
cv

CountVectorizer(max_features=10000, stop_words='english')

In [12]:
vector=cv.fit_transform(new_data['tags'].values.astype('U')).toarray()
vector.shape

(10000, 10000)

In [13]:
similarity=cosine_similarity(vector)
similarity

array([[1.        , 0.05634362, 0.12888482, ..., 0.07559289, 0.11065667,
        0.06388766],
       [0.05634362, 1.        , 0.07624929, ..., 0.        , 0.03636965,
        0.        ],
       [0.12888482, 0.07624929, 1.        , ..., 0.02273314, 0.06655583,
        0.08645856],
       ...,
       [0.07559289, 0.        , 0.02273314, ..., 1.        , 0.03253   ,
        0.02817181],
       [0.11065667, 0.03636965, 0.06655583, ..., 0.03253   , 1.        ,
        0.0412393 ],
       [0.06388766, 0.        , 0.08645856, ..., 0.02817181, 0.0412393 ,
        1.        ]])

In [14]:
new_data[new_data['title']=="The Godfather"].index[0]


2

In [15]:
distance = sorted(list(enumerate(similarity[2])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:5]:
    print(new_data.iloc[i[0]].title)

The Godfather
The Godfather: Part II
Blood Ties
Joker
Bomb City


In [16]:
def recommand(movies):
    index=new_data[new_data['title']==movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[0:5]:
        print(new_data.iloc[i[0]].title)

In [17]:
recommand("Iron Man")

Iron Man
Iron Man 3
Guardians of the Galaxy Vol. 2
Avengers: Age of Ultron
Star Wars: Episode III - Revenge of the Sith


In [18]:
import pickle

In [19]:
pickle.dump(new_data, open('movies_list.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [20]:
pickle.load(open('movies_list.pkl', 'rb'))


,id,title,rating,tags
0,278,The Shawshank Redemption,8.7,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,8.7,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,8.7,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,8.6,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,8.6,In the continuing saga of the Corleone crime f...
...,...,...,...,...
9995,10196,The Last Airbender,4.7,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,4.7,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,4.6,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,4.7,A man named Farmer sets out to rescue his kidn...


In [21]:
# Import necessary modules
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(movies, test_size=0.2, random_state=42)

# Display the sizes of the training and testing sets
print(f'Training set size: {len(train_data)}')
print(f'Testing set size: {len(test_data)}')

# Save the splits to CSV files (optional)
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

Training set size: 8000
Testing set size: 2000


In [22]:
!pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


In [24]:
import streamlit as st

# Title of the app
st.title('My Movie Recommendation System')

# Display some text
st.write('Welcome to the movie recommendation system!')

# Example of adding a simple user input
movie = st.text_input('Enter a movie you like:')

# Display the input movie
st.write(f'You entered: {movie}')


In [32]:
import streamlit as st
import pickle
import requests


movies = pickle.load(open("movies_list.pkl", 'rb'))
similarity = pickle.load(open("similarity.pkl", 'rb'))
movies_list=movies['title'].values

st.header("Movie Recommended System")
selectvalue=st.selectbox("select movie from dropdown",movies_list)

def fetch_poster(movie_id):
     url = "https://api.themoviedb.org/3/movie/{}?api_key=410b1dc5207f01165d3ecf03f4841298&language=en-US".format(movie_id)
     data=requests.get(url)
     data=data.json()
     poster_path = data['poster_path']
     full_path = "https://image.tmdb.org/t/p/w500/"+poster_path
     return full_path


def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}?api_key=410b1dc5207f01165d3ecf03f4841298&language=en-US".format(movie_id)
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        poster_path = data.get('poster_path')
        if poster_path:
            full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
            return full_path
        else:
            print("Poster path not found for movie:", movie_id)
            return None
    except requests.RequestException as e:
        print("Error fetching poster for movie:", movie_id)
        print(e)
        return None

def recommend(movie):
    index=movies[movies['title']==movie].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    recommend_movie= []
    recommend_poster=[]

    for i in distance[0:6]:
        movies_id=movies.iloc[i[0]].id
        recommend_movie.append(movies.iloc[i[0]].title)
        recommend_poster.append(fetch_poster(movies_id))
    return recommend_movie , recommend_poster

st.markdown(
    """
    <style>
    .stButton > button:first-child {
        background-color: #FF0000; /* Set the background color of the button */
        color: #FFFFFF; /* Set the text color of the button */
    }
    </style>
    """,
    unsafe_allow_html=True
)




if st.button("Show Recommend"):
    movie_name,movie_poster=recommend(selectvalue)
    col1,col2,col3,col4,col5 = st.columns(5)
    with col1:
        st.text(movie_name[0])
        st.image(movie_poster[0])
    with col2:
        st.text(movie_name[1])
        st.image(movie_poster[1])
    with col3:
        st.text(movie_name[2])
        st.image(movie_poster[2])
    with col4:
        st.text(movie_name[3])
        st.image(movie_poster[3])
    with col5:
        st.text(movie_name[4])
        st.image(movie_poster[4])


In [33]:
import requests

def get_public_ip():
    response = requests.get('https://api.ipify.org')
    return response.text

public_ip = get_public_ip()
print(f"Public IP Address: {public_ip}")

Public IP Address: 23.251.149.11


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://23.251.149.11:8501

npx: installed 22 in 4.769s
your url is: https://fresh-wolves-stare.loca.lt
